<div align="right"><i>COM418 - Computers and Music</i></div>
<div align="right"><a href="https://people.epfl.ch/paolo.prandoni">Paolo Prandoni</a>, <a href="https://www.epfl.ch/labs/lcav/">LCAV, EPFL</a></div>

<p style="font-size: 30pt; font-weight: bold; color: #B51F1F;">Everything you wanted to know about filters<br>but, wisely, never asked</p>

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import IPython

In [ ]:
plt.rcParams['figure.figsize'] = 14, 4 
DEFAULT_SF = 32000

# What is a filter?

A filter is a device (or material) used to modify an input quantity by holding back some of its components.  

## Etymologically...
<img src="img/airfilter.jpg" alt="Drawing" style="float: right; width: 500px; margin: 0px 30px;"/>

The word "filter" comes from "felt", a type of thick, porous fabric used as a sieve for liquids.


<img src="img/tips.jpg" alt="Drawing" style="float: lefti; width: 200px; margin: 30px 30px;"/>



## Acoustic filters
<img src="img/earplugs1.jpg" alt="Drawing" style="float: left; width: 300px; margin: 100px 30px;"/>
<img src="img/earplugs2.jpg" alt="Drawing" style="float: right; width: 300px; margin: 100px 30px;"/>


## Mechanical filters

<img src="img/shockabsorber.jpg" alt="Drawing" style="float: left; width: 500px; margin: 100px 30px;"/>

## Optical filters

<img src="img/3dglasses.png" alt="Drawing" style="float: left; width: 500px; margin: 100px 30px;"/>

## Electronic filters

<img src="img/crossover.jpg" alt="Drawing" style="float: left; width: 500px; margin: 100px 30px;"/>

## Digital filters

<img src="img/biquad.png" alt="Drawing" style="float: left; width: 600px; margin: 100px 30px;"/>

# Passive electronic filters

The theory of electronic filters developed at the beginning of the 20th century when engineers tried to overcome the difficulties of sending electrical signals over longer and longer wires. A long wire:
 * has an intrinsic resistence causing attenuation (understood at the time)
 * has a _distributed capacitance_ (not fully understood at the time)

The net result is that a long wire is equivalent to a RC lowpass filter.

<img src="img/rc.png" alt="Drawing" style="float: left; width: 500px; margin: 50px 300px;"/>


## The capacitor
<img src="img/capacitor.jpg" alt="Drawing" style="float: right; width: 300px; margin: 0px 30px;"/>

A capacitor is a device that can **store** energy by accumulating charges and creating an electric field

 * discovered in 1746 (the Leyden jar)
 * basically used as a battery until the development of radio

### Capacitance (Farads)

$$
    Q = CV
$$

<br>
energy stored:
$$
    E_C = \frac{1}{2}CV^2
$$

### Dynamic behavior

$$
    \frac{\partial Q}{\partial t} = C\frac{\partial V}{\partial t} \quad \Rightarrow \quad i(t) = C v'(t)
$$

### Hydraulic analogy

<img src="img/Capacitor-animation.gif" alt="Drawing" style="width: 800px; margin: 0px 30px;"/>

<i><small>Image by KDS4444 - Own work, CC BY-SA 4.0, https://commons.wikimedia.org/w/index.php?curid=64021013</small></i>

### Ohm's law for a resistor

<img src="img/resistance.jpg" alt="Drawing" style="float: right; width: 400px; margin: 0px 30px;"/>


Resistors are indifferent to frequency; Ohm's law:

$$
    \frac{v(t)}{i(t)} = R
$$



### Capacitive reactance
<img src="img/reactance.jpg" alt="Drawing" style="float: right; width: 400px; margin: 0px 30px;"/>

 * voltage across the capacitor: $v(t) = \cos(\omega t)$
 * current: $i(t) = Cv'(t) = -\omega C\sin(\omega t)$
 * _reactance_ : ratio of peak voltage to peak current for a sinusoidal input:
 $$
    X_C = \frac{1}{\omega C}
 $$

A capacitor acts as a frequency-dependent resistor:
 * at low frequencies a capacitor acts like a large resistor
 * at high frequencies a capacitor acts like a wire

### The RC lowpass filter

<img src="img/rc.png" alt="Drawing" style="float: right; width: 300px; margin: 0px 30px;"/>

differential equation:
$$
\begin{align*}
    v_\mathrm{out}(t) &= v_\mathrm{in}(t) - Ri(t) &\qquad \mbox{(voltage drop on resistor)}\\
                      &= v_\mathrm{in}(t) - RCv_\mathrm{out}'(t)  &\qquad \mbox{(current is the same on R and C)}\\
\end{align*}
$$

<br>
using the Laplace transform:

$$
    V_\mathrm{out}(s) = \frac{1}{RCs + 1}V_\mathrm{in}(s)\qquad \mbox{(transfer funcion)}\\
$$


An RC filter has an attenuation of 20 dB/decade on a Bode plot. Example for $R=10~\mathrm{k}\Omega$ and $C=100~\mathrm{nF}$:

In [ ]:
rc_lp = [1/100, 1]
plt.semilogx(*signal.bode(([1], rc_lp), w=np.logspace(0, 4, 1000))[:2]);
plt.grid();

### The RC lowpass (discretized)

pick sampling period $T$:

$$
\begin{align*}
    v_\mathrm{in}(nT) &= x[n]\\
    v_\mathrm{out}(nT) &= y[n]\\
    v_\mathrm{out}'(nT) &\approx \frac{y[n] - y[n-1]}{T}
\end{align*}
$$


and discretize $v_\mathrm{out}(t) = v_\mathrm{in}(t) - RCv_\mathrm{out}'(t)$:
$$
\begin{align*}
    y[n] &= x[n] - (RC/T)(y[n] - y[n-1]) & \qquad RC/T  \rightarrow \alpha \\
    (1+\alpha)y[n] &= x[n] + \alpha y[n-1]\\
    y[n] &= \frac{1}{1+\alpha} x[n] + \frac{\alpha}{1+\alpha} y[n-1] & \qquad \alpha/(1+\alpha) \rightarrow \lambda \\ \\
    y[n] &= (1-\lambda) x[n] + \lambda y[n-1] \qquad \leftarrow \mbox{we know this guy!}
\end{align*}
$$

In [ ]:
lam=0.9
w, h = signal.freqz(1-lam, [1, -lam], worN=np.linspace(-np.pi, np.pi, 1001))
plt.plot(w, np.abs(h))
plt.grid()

In [ ]:
plt.semilogx(w, 10 * np.log(np.abs(h)))
plt.grid()

RC circuits are passive and work by dissipating the energy of unwanted frequencies through the resistor: gain is always at most one. 


## The telegraph system
<img src="img/telegraph.jpg" alt="Drawing" style="float: right; width: 400px; margin: 0px 0px;"/>

A basic telegraph system is just an ON/OFF circuit over a long wire. First demonstrated in 1844.

Detection can be performed acoustically, visually (lamp, needle or trace on paper)

Things are simple as long as the wire is not too long...

### Let's build a Morse-code transmitter

In [ ]:
def morse(msg, wpm=50, sf=DEFAULT_SF):
    # using PARIS timing values:
    DIT = int(sf * 60 / (50 * wpm)) # dot
    DAH = 3 * DIT  # dash
    ISS = 1 * DIT  # intra-symbol space
    ICS = 3 * DIT  # inter-character space
    IWS = 7 * DIT  # inter-word space
    DICT = { 
        'A':'.-', 'B':'-...','C':'-.-.', 'D':'-..', 'E':'.', 'F':'..-.', 'G':'--.', 'H':'....', 'I':'..', 'J':'.---', 
        'K':'-.-', 'L':'.-..', 'M':'--', 'N':'-.', 'O':'---', 'P':'.--.', 'Q':'--.-', 'R':'.-.', 'S':'...', 'T':'-',
        'U':'..-', 'V':'...-', 'W':'.--', 'X':'-..-', 'Y':'-.--', 'Z':'--..', '1':'.----', '2':'..---', '3':'...--', 
        '4':'....-', '5':'.....', '6':'-....', '7':'--...', '8':'---..', '9':'----.', '0':'-----', ', ':'--..--', 
        '.':'.-.-.-', '?':'..--..', '/':'-..-.', '-':'-....-', '(':'-.--.', ')':'-.--.-'}
    
    # crude upper bound on number of samples
    x, ix = np.zeros(30 * DIT * len(msg), dtype=float), 0
    for c in msg.upper():
        if c == ' ':
            ix += IWS
        else:
            for d in DICT[c]:
                h = DIT if d == '.' else DAH
                x[ix:ix+h] = 1
                ix = ix + h + ISS
            ix += ICS
    return np.linspace(0, ix / sf, ix), x[:ix]

In [ ]:
t = morse("computers and music", wpm=25)
plt.plot(*t);

In [ ]:
IPython.display.Audio(t[1], rate=DEFAULT_SF)

In [ ]:
from IPython.display import YouTubeVideo
id='YPsgEdmlUf0?t=12'
YouTubeVideo(id=id,width=600,height=300)

### Sending telegrams from Europe to America

In the 1860 the first telegraph cables were laid between Europe and the US (~3000 km)

A long cable is like an RC lowpass filter:
 * $\approx 100~\Omega/\mathrm{km}$ resistance
 * $\approx 1~\mathrm{nF}/\mathrm{km}$ capacitance
 
For a 3000 km cable, equivalent cutoff frequency $\approx 0.2~\mathrm{Hz}$

### Simulating the RC lowpass with a leaky integrator

Useful approximation for a leaky integrator with cutoff $f_c$ working at $F_s$ samples per second:

$$
    \lambda \approx e^{-2\pi f_c/F_s}
$$

### See what happens at the other end

In [ ]:
def test_wpm(wpm, fc=0.2, sf=DEFAULT_SF, msg="computers and music"):
    n, t = morse(msg, wpm=wpm)
    lam = np.exp(-2 * np.pi * fc / DEFAULT_SF)
    rt = signal.lfilter([1-lam], [1, -lam], t)
    plt.plot(n, t, 'gray')
    plt.plot(n, rt)
    return rt

In [ ]:
rt = test_wpm(25)

In [ ]:
IPython.display.Audio(rt, rate=DEFAULT_SF)

### How slow do we have to transmit?

In [ ]:
test_wpm(10);

In [ ]:
test_wpm(5);

In [ ]:
test_wpm(1);

### How do we fix this?

To widen the bandwidth of the transmission line we need to "pull up" the transfer function. How can we do it?

 * the modern way: use amplifiers and build **active** filters with gains > 1
 * in the early days: use **inductors** and build passive RLC filters

## The inductor

<img src="img/inductor.jpg" alt="Drawing" style="float: right; width: 300px; margin: 0px 30px;"/>

An inductor is an electronic device that can **store** energy by creating a magnetic field.

$$ E_L = \frac{1}{2}LI^2 $$

### Dynamic behavior

An inductor tries to "resist" changes in current flow by generating an opposing voltage:

$$
    v(t) = -L i'(t)
$$

### Hydraulic analogy

<img src="img/inductor.png" alt="Drawing" style="width: 400px; margin: 100px 0 0 300px;"/>


### Inductive reactance
<img src="img/inductance.jpg" alt="Drawing" style="float: right; width: 400px; margin: 0px 30px;"/>

 * voltage across inductor: $v(t) = \cos(\omega t)$
 * current: $i(t) = \sin(\omega t)/ (\omega L)$
 * _reactance_ : ratio of peak voltage to peak current for a sinusoidal input:
    $$
        X_L = \omega L
    $$


An inductor also acts as a frequency-dependent resistor:
 * at low frequencies an inductor acts like a wire
 * at high frequencies an inductor acts like a large resistor
 * miraculously, this is the inverse behavior of a capacitor (thank you Maxwell!)
 
OK, but how does this help us "lift up" the frequency response?

### Intuition #1: the kickback

<img src="img/kickback.jpg" alt="Drawing" style="float: left; height: 200px; margin: 50px 60px;"/>

 * close circuit: current starts to increase from zero with inital slope $V$
 * steady state: voltage across inductor is zero and current is $i_0$ (finite because of some resistance)
 * open circuit: current drops to zero instantaneously; voltage across inductor: $v(t) \approx -L i_0/\Delta t \rightarrow -\infty$
 
 
(Of course we can increase the voltage but not the power, since inductors are passive.)

### Intuition #2: resonance in an LC circuit (tank circuit)

<img src="img/lctank.jpg" alt="Drawing" style="height: 400px;"/>





 * natural frequency of the system $\omega_0 = 1/\sqrt{LC}$
 * amplitude of the oscillation: depending on the amount of initial charge
 * if a little charge is repetedly added _at the right times_ , amplitude increases indefinitely because no energy is lost
 * of course in reality there are losses

### Hydraulic analogy for the LC tank circuit

<img src="img/LChydraulic.png" alt="Drawing" style="width: 400px; margin: 0px 30px;"/>


### Digression: Resonance

How can a physical system store energy? Static methods:
  * as chemical bonds (batteries, food, fuel)
  * as potential energy (dams)
  * as thermal energy (liquid gas)
  * ...
 
 
A **resonant system** stores energy by alternately transferring it between different forms of storage:
  * pendulum (kinetic vs. potential)
  * vibrating string (kinetic vs. tensional)
  * LC circuit (electric vs magnetic field)
  * ...

#### Resonance frequency

The cadence of the internal energy transfer is called the natural frequency of a resonant system. 

Input energy at or near the resonant frequency will be absorbed by the system easily (and conversely). 

#### Resonance and complex poles

The transfer function of a LTI resonant system necessarily has complex poles. Why?

  * the impulse response of a LTI system is always a linear combination of decaying exponentials 
  * each exponential corresponds to a pole
  * the resonant mechanism has an oscillatory component: the exponential must be complex

### The RLC lowpass filter

<img src="img/rlc.jpg" alt="Drawing" style="float: right; width: 400px; margin: 0px 30px;"/>

$$
    V_\mathrm{out}(s) = \frac{1}{LCs^2 + RCs + 1}V_\mathrm{in}(s)
$$

 * second order filter
 * resonant frequency $\omega_0 = 1/\sqrt{LC}$
 * peak amplitude $Q = \frac{1}{R}\sqrt{\frac{L}{C}}$
 * no peak for $Q \le 1/\sqrt{2}$

 * -40 dB/decade (twice the rolloff of an RC)
 * resonance peak if poles are complex-valued

In [ ]:
plt.semilogx(*signal.bode(([1], [(1/100)**2, 1/40, 1]), w=np.logspace(0, 4, 1000))[:2]);
plt.semilogx(*signal.bode(([1], [(1/100)**2, 1/70, 1]), w=np.logspace(0, 4, 1000))[:2]);
plt.semilogx(*signal.bode(([1], [(1/100)**2, 1/200, 1]), w=np.logspace(0, 4, 1000))[:2]);
plt.semilogx(*signal.bode(([1], [(1/100)**2, 1/1000, 1]), w=np.logspace(0, 4, 1000))[:2]);
plt.grid();

### "Pulling up" a transmission line

How to use a RLC lowpass to double the bandwidth of a fixed RC-like transmission line:

In [ ]:
rlc_lp = [1/30000, 1/300, 1]
plt.semilogx(*signal.bode(([1], rc_lp), w=np.logspace(0, 4, 1000))[:2], label="RC lowpass");
plt.semilogx(*signal.bode(([1], rlc_lp), w=np.logspace(0, 4, 1000))[:2], label="RLC");
# cascading filters multiplies the transfer functions, which is equivalent to polynomial multiplication
plt.semilogx(*signal.bode(([1], np.polymul(rc_lp, rlc_lp)), w=np.logspace(0, 4, 1000))[:2], label="cascade");
plt.grid();
plt.legend();

In [ ]:
plt.semilogx(*signal.bode(([1], rc_lp), w=np.logspace(0, 4, 1000))[:2]);
plt.semilogx(*signal.bode(([1], np.polymul(rc_lp, rlc_lp)), w=np.logspace(0, 4, 1000))[:2], 'C2');
plt.xlim(10, 500)
plt.ylim(-10, 2)
plt.grid();

In practice, inductors called **loading coils** were spliced up every few km in telephone lines to flatten the bandwidth. As a side effect, traditional telephone lines have a cutoff at 4 kHz.

<img src="img/loadingcoil.jpg" alt="Drawing" style="float: left; height: 200px; margin: 0px 0px;"/>


### The RLC second-order sections

A RLC filter can be designed to be:
  * lowpass
  * highpass
  * bandpass (in the limit: resonator)
  * bandstop (int the limit: notch)
  * allpass
  
Second order sections can be cascaded to implement higher-order filters. But passive analog sections are not decoupled, so it's a bit tricky.

## Active Electronic Filters

Active filters use amplifying elements (tubes, transistors, op-amps, etc)

 * no need for inductors 
 * gain larger than unity
 * decoupling between cascaded sections
 
But
 * need power supply
 * limited to low-voltage applications (eg, still need passive filters in loudspeaker crossover filters)

### The active second-order lowpass

<img src="img/activelowpass.jpg" alt="Drawing" style="float: right; width: 400px; margin: 0px 30px;"/>

$$
    V_\mathrm{out}(s) = \frac{1}{a_2 s^2 + a_1 s + 1}V_\mathrm{in}(s)
$$

 * second order filter
 * resonant frequency $\omega_0 = 1/\sqrt{R_1 R_2 C_1 C_2}$
 * peak amplitude $Q = \sqrt{R_1 R_2 C_1 C_2}/(C_2(R_1 + R_2))$
 * no peak for $Q \le 1/\sqrt{2}$

### The active second-order section

<img src="img/towthomas.jpg" alt="Drawing" style="float: right; width: 400px; margin: 0px 30px;"/>

$$
    V_\mathrm{out}(s) = \frac{b_2 s^2 + b_1 s + b_0}{a_2 s^2 + a_1 s + 1}V_\mathrm{in}(s)
$$

Very complicated relation between transfer function coefficients and component ratings. 

<img src="img/biquad.png" alt="Drawing" style="float: right; width: 400px; margin: 20px 30px;"/>

# Luckily, we prefer DSP!

Digital biquad sections provide:
 * the ability to replicate every analog designs 
 * all the advantages of active filters 
 * superior precision (no discrete components)
 * complete decoupling in cascade realizations
 * fully customizable second-order transfer functions